In [252]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
import json
import re
from bs4 import BeautifulSoup

In [4]:
def remove_prefix_from_name(name):
    pattern = r"[a-z]{2,}\.\s"
    prefixes = re.findall(pattern, name, flags=re.IGNORECASE)
    
    if prefixes:
        name_without_prefixes = re.sub(pattern, '', name, flags=re.IGNORECASE).strip()
        
        if 'IJ. ' in prefixes:
            prefixes = [i for i in prefixes if 'IJ.' not in i]
            name_without_prefixes = 'IJ. ' + name_without_prefixes
        
        return prefixes, name_without_prefixes
    else:
        return '', name

In [5]:
def xpath(element, path, namespaces):
    obj = element.find(path, namespaces)
    
    if obj is not None:
        return obj.text
    else:
        return ''

In [222]:
def xpath_TOOI(element, path, namespaces, afkorting):
    
    dc_publisher = element.find(path, namespaces)
    if dc_publisher is not None:

        # hier wordt de afkorting gebruikt bijv. ws = waterschappen
        if dc_publisher.text.rsplit('/')[-1][:len(afkorting)] == afkorting:
            
            return dc_publisher.text.rsplit('/')[-1]
        else:
            return ''
    else:
        return ''
    

Uitdagingen namen: prefixes als Ch. Chr. Th. Ph. zijn eigenlijk initialen. Wat te doen met initialen zonder puntjes. 

In [223]:
def xpath_naam(element, path, namespaces):
    # verzamel de naam in de XML
    initial_name = element.find(path, namespaces)
    
    if initial_name is not None:
        initial_name_txt = initial_name.text

        # er staat vaak een prefix als dr. of mew. voor deze wordt verwijderd?
        prefixes, name_without_prefix = remove_prefix_from_name(initial_name_txt)
        name_without_prefix = name_without_prefix.lstrip()

        initial_name_split = name_without_prefix.split(' ', 1)

        # bij gemeenten staat er een persoon in als mew. Schouten dus zonder initialen. 
        if len(initial_name_split) == 2:
            first_word = initial_name_split[0]
            if '.' not in first_word and len(first_word) != 1:

                foaf_firstName = initial_name_split[0]
                foaf_initials = foaf_firstName[0].upper() + '.'
            else:
                foaf_initials = initial_name_split[0]
                foaf_firstName = ''

            foaf_lastName = initial_name_split[1]
        
        else:
            foaf_initials = ''
            foaf_firstName = ''
            foaf_lastName = initial_name_split[0]
    
    # gebeurt in de praktijk niet maar maakt het wel failproof
    else:
        foaf_initials = ''
        foaf_firstName = ''
        foaf_lastName = ''
        
    prefixes = ''.join(prefixes)
    prefixes = prefixes.rstrip()   
            
    return foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes

TODO: van tree automatisch naar namespaces gaan.

In [320]:
def json_create_single_layer(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',}
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]


    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)
    
    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)

        # in het functie_element staan alle verschillende functies
        functie_element = organisatie.findall('p:functies/p:functie', namespaces)

        # loop over deze functies
        for functie in functie_element:

            foi_count = 0
            DICT = {}

            foaf_function_type = xpath(functie, 'p:naam', namespaces)

            # vindt alle medewerkers die bij deze functie horen
            medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

            # loop over alle medewerkers om per medewerker gegevens te verzamelen
            for medewerker in medewerker_element:
                
                foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)
                
                
                # optioneel startdatum, telefoonnummer en mailadres
                foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
                
                # genereer beschikbaarheidsgegevens
                if organisatie_type == 'Waterschap':
                    bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {Type.lower()} {dc_publisher_name}"
                else:
                    if foi_party != '':
                        bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                    else:
                        bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"
                
                query = name_without_prefix + ' ' + foaf_function_type + ' ' + dc_publisher_name
                
                hrefs = search_yahoo(query)
    
                foi_twitter, foi_website, foi_linkedin = extract_urls(hrefs, website_txt, foaf_lastName)
            
                # vul de dictionary in
                Dict = {
                        'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                        'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                        'foi_function': foaf_function_type,
                        'dc_description': bereikbaarheidsgegevens,
                        'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{organisatie_type.lower()}.xml",
                        'dc_publisher': dc_publisher,
                        'dc_creator': "Ramon Duursma",
                        'foi_retrievedDate': foi_retrievedDate,
                        'dc_date_year': dc_date_year,
                        'dc_publisher_name': dc_publisher_name,
                        'foaf_initials': foaf_initials,
                        'foaf_firstName': foaf_firstName,
                        'foaf_lastName': foaf_lastName,
                        'foi_title': prefixes,
                        'foaf_name': name_without_prefix,
                        'foaf_mbox': foaf_mbox,
                        'foaf_phone': foaf_phone,
                        'foi_twitter': foi_twitter,
                        'foi_website': foi_website,
                        'foi_linkedin': foi_linkedin,
                        'foaf_type': Type,
                        'foi_startDate': foi_startDate,
                        'foi_party': foi_party,
                        'foi_files': [] ,
                    }

                # Verwijder alle lege strings uit de dict
                filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                # en voeg de dictionary toe aan 
                DICT[foi_count] = filtered_dict

                foi_count+=1

            # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
            final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                        'infobox': {'foi_totalDossiers': len(DICT),
                                     'foi_dossiers': DICT}}


            list_of_dicts.append(final_dict)

    number_of_people = sum(i['infobox']['foi_totalDossiers'] for i in list_of_dicts)
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts


In [321]:
organisatie = 'Waterschap'
people_num, dict_list = json_create_single_layer(organisatie)

tree = ET.parse(f"exportOO_full.xml")
namespaces = {
    'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)
list_of_peoples = [organisatie_element.findall('.//p:medewerker', namespaces) for organisatie_element in organisatie_elements]
people = sum(len(lst) for lst in list_of_peoples)

print(f'aantal medewerkers in XML: {people}')
print(f'aantal medewerkers extracted in JSON: {people_num}')


identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foi_twitter', 'foi_website', 'foi_linkedin', 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

for dl in dict_list:
    dossiers = dl['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1

print(id_dict)

C:\Users\ramon\AppData\Local\Temp\ipykernel_20168\4189120594.py:25: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Waterschap"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


aantal medewerkers in XML: 42
aantal medewerkers extracted in JSON: 42
{'foi_function': 42, 'dc_publisher': 42, 'foaf_initials': 42, 'foaf_firstName': 1, 'foaf_lastName': 42, 'foi_title': 22, 'foaf_mbox': 2, 'foaf_phone': 2, 'foi_twitter': 0, 'foi_website': 16, 'foi_linkedin': 19, 'foaf_type': 42, 'foi_startDate': 25, 'foi_party': 0}


C:\Users\ramon\AppData\Local\Temp\ipykernel_20168\3243492678.py:9: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Waterschap"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)


In [322]:
def json_create_multi_layer(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Provincies': 'pv',
                  'Ministerie': 'mnre'}
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse(f"exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)

    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url''p:types/p:type', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        organisatie_elements2 = organisatie.findall('.//p:organisaties/p:organisatie', namespaces)
        
        for organisatie2 in organisatie_elements2:
            
            organisatie_naam = xpath(organisatie2, 'p:naam', namespaces)
        
            # in het functie_element staan alle verschillende functies
            functie_element = organisatie2.findall('p:functies/p:functie', namespaces)


            # loop over deze functies
            for functie in functie_element:
                foi_count = 0
                DICT = {}

                foaf_function_type = xpath(functie, 'p:naam', namespaces)

                # vindt alle medewerkers die bij deze functie horen
                medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

                # loop over alle medewerkers om per medewerker gegevens te verzamelen
                for medewerker in medewerker_element:

                    foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                    foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)


                    # optioneel startdatum, telefoonnummer en mailadres
                    foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                    foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                    foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)

                    # genereer beschikbaarheidsgegevens

                    if organisatie_type != 'Provincies':
                        if foi_party != '':
                            bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                        else:
                            bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"

                    # vul de dictionary in
                    Dict = {
                            'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                            'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                            'foi_function': foaf_function_type,
                            'dc_description': bereikbaarheidsgegevens,
                            'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{organisatie_type.lower()}.xml",
                            'dc_publisher': dc_publisher,
                            'dc_creator': "Ramon Duursma",
                            'foi_retrievedDate': foi_retrievedDate,
                            'dc_date_year': dc_date_year,
                            'dc_publisher_name': dc_publisher_name,
                            'foaf_initials': foaf_initials,
                            'foaf_firstName': foaf_firstName,
                            'foaf_lastName': foaf_lastName,
                            'foi_title': prefixes,
                            'foaf_name': name_without_prefix,
                            'foaf_mbox': foaf_mbox,
                            'foaf_phone': foaf_phone,
                            'foaf_type': Type,
                            'foi_startDate': foi_startDate,
                            'foi_party': organisatie_naam,
                            'foi_files': [] ,
                        }

                    # Verwijder alle lege strings uit de dict
                    filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                    # en voeg de dictionary toe aan 
                    DICT[foi_count] = filtered_dict

                    foi_count+=1

                # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                            'infobox': {'foi_totalDossiers': len(DICT),
                                         'foi_dossiers': DICT}}


                list_of_dicts.append(final_dict)

    number_of_people = sum(i['infobox']['foi_totalDossiers'] for i in list_of_dicts)
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts

In [323]:
people_num, dict_list = json_create_multi_layer('Ministerie')

organisatie = 'Ministerie'

tree = ET.parse(f"exportOO_full.xml")
namespaces = {
    'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)
list_of_peoples = [organisatie_element.findall('.//p:medewerker', namespaces) for organisatie_element in organisatie_elements]
people = sum(len(lst) for lst in list_of_peoples)

print(f'aantal medewerkers in XML: {people}')
print(f'aantal medewerkers extracted in JSON: {people_num}')


identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

for dl in dict_list:
    dossiers = dl['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1

print(id_dict)

C:\Users\ramon\AppData\Local\Temp\ipykernel_20168\1478380439.py:24: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Ministerie"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


aantal medewerkers in XML: 741
aantal medewerkers extracted in JSON: 688
{'foi_function': 688, 'dc_publisher': 688, 'foaf_initials': 640, 'foaf_firstName': 174, 'foaf_lastName': 688, 'foi_title': 393, 'foaf_mbox': 176, 'foaf_phone': 273, 'foaf_type': 688, 'foi_startDate': 101, 'foi_party': 688}


C:\Users\ramon\AppData\Local\Temp\ipykernel_20168\856358615.py:10: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Ministerie"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)


In [25]:
D = {}

for i in dict_list:
    x = i['infobox']['foi_dossiers'][0]['dc_publisher_name']
    y = i['infobox']['foi_totalDossiers']
    if x not in D:
        D[x] = y
    else:
        D[x] += y
    
print(D)
print(sum(D.values()))
print(len(D))

{'Provincie Drenthe': 3, 'Provincie Flevoland': 3, 'Provincie Fryslân': 6, 'Provincie Groningen': 92, 'Provincie Noord-Holland': 65, 'Provincie Overijssel': 77, 'Provincie Zuid-Holland': 93}
339
7


In [308]:
def search_yahoo(Query):

    time.sleep(1)
    
    url = "http://search.yahoo.com/search?p=%s"
    r = requests.get(url % Query) 
    soup = BeautifulSoup(r.text)
    hrefs = [a['href'] for a in soup.find_all('a', href=True)]
    hrefs = [href for href in hrefs if 'yahoo' not in hrefs]

    new_hrefs = [href for href in hrefs if "yahoo" not in href and '#' not in href]
    
    return new_hrefs

In [319]:
print(new_hrefs)

['https://www.aaenmaas.nl/bestuur/directieteam/pieter-sennema/', 'https://cc.bingj.com/cache.aspx?q=P.+Sennema+Dijkgraaf+Aa+en+Maas&d=4964114863164080&mkt=en-US&setlang=en-US&w=2eONpfA4Q97Ra28bkYc4-pUlkpg3ex9v', 'https://www.aaenmaas.nl/publish/pages/955/2022-06-20-organigram_waterschap_aa_en_maas.pdf', 'https://organisaties.overheid.nl/41917/P._Sennema/', 'https://cc.bingj.com/cache.aspx?q=P.+Sennema+Dijkgraaf+Aa+en+Maas&d=4697831187038899&mkt=en-US&setlang=en-US&w=SaQXpEk1pqRYU8qSBcI_2hP4mUsfKCWY', 'https://nl.linkedin.com/in/pietersennema', 'https://www.gelderlander.nl/den-bosch/mario-jacobs-gestart-als-nieuwe-dijkgraaf-van-aa-en-maas~ad5bee15/202259459/', 'https://www.bestebestuurder.nl/genomineerden/dijkgraaf-verheijen-waterschap-aa-en-maas/', 'https://www.waterforum.net/mario-jacobs-geinstalleerd-als-dijkgraaf-aa-en-maas/', 'https://www.gelderlander.nl/cuijk/mario-jacobs-nieuwe-dijkgraaf-aa-en-maas~ac34aa195/', 'https://zoek.officielebekendmakingen.nl/wsb-2016-9265.pdf', 'https:/

In [309]:
def extract_urls(hrefs, website_url, last_name):
    
    URLs = {'twitter_url': '', 'account_url': '', 'linkedin_url': ''}
    
    last_name = last_name.lower()
    
    twitter = False
    website = False
    linkedin = False
    
    for url in hrefs:
        
        url = url.lower()
        if 'twitter' in url and last_name in url and twitter == False:
            twitter = True
            URLs['twitter_url'] = url
            
        if website_url in url and last_name in url and website == False:
            website = True
            URLs['account_url'] = url

        if 'linkedin' in url and last_name in url and linkedin == False:
            linkedin = True
            URLs['linkedin_url'] = url
    
    return URLs['twitter_url'], URLs['account_url'], URLs['linkedin_url']
        
TESTER = extract_urls(new_hrefs, 'www.aaenmaas.nl', 'sennema')
print(TESTER)

('', 'https://www.aaenmaas.nl/bestuur/directieteam/pieter-sennema/', 'https://nl.linkedin.com/in/pietersennema')


In [26]:
%%timeit

json_create_provincies('Provincies')

C:\Users\ramon\AppData\Local\Temp\ipykernel_16304\1279882358.py:23: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './p:organisaties/p:organisatie'
  organisatie_elements = tree.findall('/p:organisaties/p:organisatie', namespaces)


29.7 ms ± 233 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
